### Analisis de grupo de Whatsapp

In [ ]:
#Autor: CARF

In [ ]:
# Cargar Librerias

import pandas as pd
import re
import numpy as np
import collections
import emoji
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
#Funciones de recuperación y limpieza

def starts_with_date_time(s):
    # regex pattern for date
    pattern = r'^\d{1,2}/\d{1,2}/\d{4}\s\d{1,2}:\d{2}\s[ap].\sm\.'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
# Se reconoce el nombre de usuario en cualquier formato registrado.
def find_author(s):
    patterns = [
        '([\w]+):',                        # Primer Nombre
        '([\w]+[\s]+[\w]+):',              # Primer Nombre + Apellido
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # Primer Nombre + Seg. nombre + Apellido
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Teléfono México
        '[\w]+ ?[^\s\u1f300-\u1f5ff]:',    # Nombre y Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
def get_data_point(line):   
    split_line = line.split(' - ') 
    date_time = split_line[0].split(' ') 
    date = date_time[0]
    time = ' '.join(date_time[1:])
    message = ' '.join(split_line[1:])
    if find_author(message): 
        split_message = message.split(': ') 
        author = split_message[0] 
        message = ' '.join(split_message[1:])
    else:
        author = None
    return date, time, author, message

In [ ]:
#Genera una lista con los datos para convertilos en data frame

parsed_data = [] 
conversation_path = "Chat.txt" # chat file
with open(conversation_path, encoding='utf8') as fp:
    fp.readline() # Skipping first line of the file 
    message_buffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if starts_with_date_time(line): 
            print(line)
            if len(message_buffer) > 0: 
                parsed_data.append([date, time, author, ' '.join(message_buffer)]) 
            message_buffer.clear() 
            date, time, author, message = get_data_point(line) 
            message_buffer.append(message) 
        else:
            message_buffer.append(line)


In [ ]:
# Genera el dataframe

df = pd.DataFrame(parsed_data, columns=['Date', 'Time', 'Author', 'Message'])
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
df.info()
df

### Configurando el Data Frame

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format= "%d/%m/%y") # Formato de fecha
df['Message'] = df['Message'].str.lower() # Mensajes en minúsculas
df['Message'] = df.Message.str.replace(r"(a|j)?(ja)+(a|j)?", "jaja") # Estandarizamos la expresión de risa a "jaja"
df = df.dropna() # Para eliminar entradas en blanco

### Anonimizando Usuarios

In [ ]:
# Lista con los personajes del Dc
got_dt = pd.read_csv("DCpersonajes.csv") 

# Obtiene nombres y alias
nombres = list(df.Author.unique())
aliases = list(got_dt.name.sample(len(nombres)))

df.Author.replace(nombres, aliases, inplace=True) # Remplaza los autores por un alias

# Remplaza las menciones de los miembros del grupo en la columna de mensajes en cada entrada en forma de string (por partes)
for (nombre, alias) in zip(nombres, aliases):
    df.Message = df.Message.str.replace(nombre, alias)
    
df.info()
df.head()


In [ ]:
df

In [ ]:
import regex
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

Autores= df.Author.unique()
total_authors= Autores.shape[0]
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<multimedia omitido>'].shape[0]
df["emoji"] = df["Message"].apply(split_count)
emojis = sum(df['emoji'].str.len())
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: regex.findall(URLPATTERN, x)).str.len()
links = np.sum(df.urlcount)

### Estadísticas Generales

In [ ]:
print("Estadísticas Generales del grupo")
print("La Cantidad de Autores del grupo es de ", total_authors,", y el total de Mensajes es de ", total_messages,".")
print("El Promedio de Mensajes enviados es de ",media_messages,".")
print("La cantidad de Emojis enviados es de ",emojis,", y el total de enlaces es de",links,".")

### Estadisticas por autor

In [ ]:
media_messages_df = df[df['Message'] == '<multimedia omitido>']
messages_df = df.drop(media_messages_df.index)

In [ ]:
messages_df['Letter_Count'] = messages_df['Message'].apply(lambda s : len(s))
messages_df['Word_Count'] = messages_df['Message'].apply(lambda s : len(s.split(' ')))
messages_df["MessageCount"]=1

In [ ]:
messages_df.head()

In [ ]:
l = messages_df.Author.unique()

for i in range(len(l)):
  # Filtering out messages of particular user
  req_df= messages_df[messages_df["Author"] == l[i]]
  # req_df will contain messages of only one particular user
  print(f'Estadísticas de {l[i]} -')
  # shape will print number of rows which indirectly means the number of messages
  print('Mensajes enviados:', req_df.shape[0])
  #Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
  print('Palabras por mensaje:', words_per_message)
  #media conists of media messages
  media = media_messages_df[media_messages_df['Author'] == l[i]].shape[0]
  print('Contenido multimedia enviado:', media)
  # emojis conists of total emojis
  emojis = sum(req_df['emoji'].str.len())
  print('Emojis enviados:', emojis)
  #links consist of total links
  links = sum(req_df["urlcount"])   
  print('Links enviados:', links)   
  print()

### Estadisticas de Emojis

In [ ]:
#Total de Emojis distintos enviados dentro de la conversación
total_emojis_list = list(set([a for b in messages_df.emoji for a in b]))
total_emojis = len(total_emojis_list)
print('Total de Emojis diferentes enviados:',total_emojis)

### Emoji mas utilizado en el grupo

In [ ]:
import collections
total_emojis_list = list([a for b in messages_df.emoji for a in b])
emoji_dict = dict(collections.Counter(total_emojis_list))
emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)

emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
emoji_df

In [ ]:
import plotly.express as px
fig = px.pie(emoji_df, values='count', names='emoji',
             title='Distribución por proporcón de Emojis')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# Crear lista de autores para crear gráficos de uso dee emoji's para cada autor
l = messages_df.Author.unique()
for i in range(len(l)):
  dummy_df = messages_df[messages_df['Author'] == l[i]]
  total_emojis_list = list([a for b in dummy_df.emoji for a in b])
  emoji_dict = dict(collections.Counter(total_emojis_list))
  emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
  print('Emoji Distribution for', l[i])
  author_emoji_df = pd.DataFrame(emoji_dict, columns=['emoji', 'count'])
  fig = px.pie(author_emoji_df, values='count', names='emoji')
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

In [ ]:
autor_df = messages_df.groupby("Author").sum()
autor_df

### Miembros con mayor actividad dentro del grupo.

In [ ]:
messages_df['Author'].value_counts().head(15).plot.barh() 
plt.xlabel('Número de mensajes')
plt.ylabel('Autor')

### Actividad dentro de la linea de tiempo

In [ ]:
date_df = messages_df.groupby("Date").sum()
date_df.reset_index(inplace=True)
fig = px.line(date_df, x="Date", y="MessageCount", labels={'Date':'Periodo', 'MessageCount':'No.Mensajes'}, title='Variación de la Actividad dentro del periodo.')
fig.update_xaxes(nticks=20)
fig.show()

### Días de la semana con mayor actividad

In [ ]:
def dayofweek(i):
  l = ["Lunes", "Martes", "Miércoes", "Jueves", "Viernes", "Sábado", "Domingo"]
  return l[i];
day_df=pd.DataFrame(messages_df["Message"])
day_df['day_of_date'] = messages_df['Date'].dt.weekday
day_df['day_of_date'] = day_df["day_of_date"].apply(dayofweek)
day_df["messagecount"] = 1
day = day_df.groupby("day_of_date").sum()
day.reset_index(inplace=True)

fig = px.line_polar(day, r='messagecount', theta='day_of_date', line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0,6000]
    )),
  showlegend=False
)
fig.show()

### Los 10 días de mayor actividad 

In [ ]:
messages_df['Date'].value_counts().head(10).plot.barh()
plt.xlabel('Número de mensajes')
plt.ylabel('Fecha')

### Las horas del día con mayor frecuencia de mensajes

In [ ]:
messages_df['Time'].value_counts().head(10).plot.barh() 
plt.xlabel('Número de mensajes')
plt.ylabel('Hora')

In [ ]:
messages_df.iloc[messages_df['Word_Count'].argmax()]

In [ ]:
text = " ".join(review for review in messages_df.Message)
print ("Hay {} palabras en todos los mensajes.".format(len(text)))

### Generar una nube de palabras

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from wordcloud import WordCloud 
stopwords = set(stopwords.words('spanish', 'english')) 
stopwords.update([ "chingo", "xq","aaay","Jajajaa","perra","kieren","verga","inés"])
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

In [ ]:
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(40, 30))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off");

In [ ]:
# Generate word cloud
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Pastel1', collocations=False, stopwords = stopwords).generate(text)
# Plot
plot_cloud(wordcloud)